# 4 Pandas

When dealing with numeric matrices and vectors in Python, NumPy makes life a lot easier. However, those used to work with dedicated languages like [R](https://www.r-project.org/), doing data analysis directly with NumPy feels like a step back. Fortunately, some nice folks have written the Python Data Analysis Library (a.k.a. [Pandas](http://pandas.pydata.org/)). Pandas provides an R-like DataFrame, produces high quality plots with matplotlib, and integrates nicely with other libraries that expect NumPy arrays.

Pandas works with `Series` of data, that then are arranged in `DataFrame` objects. A dataframe is the object closest to an Excel spreadsheet that we will see throughout the course. Dataframes, though, given that they are integrated in Python and can be combined with so many different packages, are much more powerful than simple Excel spreadsheets. The data in the series can be either qualitative or quantitative data and creating a series is as easy as creating a NumPy array from a one-dimensional list.

In [ ]:
import numpy as np
import pandas as pd
print('Pandas:', pd.__version__)

In [ ]:
animals = ['Tiger', 'Bear', 'Moose']
pd.Series(animals)

In [ ]:
numbers = [1, 2, 3]
pd.Series(numbers)

Notice that the series is indexed by default by integers. We can change this indexing by using a dictionary instead of a list to create the series.

In [ ]:
sports = {'Archery': 'Bhutan',
          'Golf': 'Scotland',
          'Sumo': 'Japan',
          'Taekwondo': 'South Korea'}
s = pd.Series(sports)
s

On the other hand, dataframes can be built from two-dimensional arrays, with the ability of labelling columns and indexing the rows. **Every column in a dataframe is a series**. 

In [ ]:
# Sampling a 1000 rows 6 cols 2D array from the standard normal distribution and creating DataFrame
u = pd.DataFrame(np.random.randn(1000, 6),
                 index=np.arange(0, 3000, 3),
                 columns=['A', 'B', 'C', 'D', 'E', 'F'])

print(type(u))

u

As you might have noticed, it is not the best to look at massive dataframes. There are some functions that allow us to have a nicer look at parts of the dataframe to have an idea of "how things are going".

In [ ]:
u.head()

In [ ]:
u.tail()

In [ ]:
u.info()

In [ ]:
u.describe()

### Indexing/Slicing in Pandas

The easiest way to access information in a Pandas dataframe, equivalent to the way used in NumPy, is using the `iloc` command. With `iloc` we can use the same indexing techniques that we saw with NumPy in the previous notebook.

In [ ]:
# Slice-in rows index 125 to 132 (132 included!) from columns index 0, 2 and 5
u.iloc[125:132, [0, 2, 5]]

We can choose specific columns according to their names using `loc` instead of `iloc`.

In [ ]:
# Slice-in rows 375 to 393 (393 included!) from columns A, C and F
u.loc[375:393, ['A', 'C', 'F']]

However, there are a few different ways of accessing the data in a Pandas dataframe, that typically have a more "direct" connection with the actual content fo the dataframe. Individual or sets of columns can also be accessed by their column names. Choosing one single column will give a Series, while two or more will produce a DataFrame

In [ ]:
u['A'].head()

In [ ]:
u[['A', 'D']].head()

Not only that, we can access a single column without the need of brackets []

In [ ]:
u.A.head()

Or, we can retrieve the elements that satisfy some condition

In [ ]:
u[u.D > 2]

Dataframes provide the `query` functionality for the same purpose. While it is less powerful than boolean indexing, it is often faster and shorter (when names are longer than just `u`)

In [ ]:
u.query('D > 2')

### Reshaping `DataFrame`

We can reshape and concatenate dataframes in a pretty similar way to numpy arrays. 

In [ ]:
df1 = pd.DataFrame()

df1['sample'] = ['A', 'A', 'A', 'B', 'B', 'B']
df1['replicate'] = ['01', '02', '03', '01', '02', '03']
df1['protein'] = 'P02768'
df1['value1'] = np.random.randn(6)

df1

In [ ]:
pivot_df1 = df1.pivot(index='replicate', columns='sample', values='value1')

pivot_df1.head()

### Computing With `DataFrames`

We can calculate with `DataFrames` or their columns (which are `Series`) the same way we would work with numpy arrays.

In [ ]:
df1['value2'] = 1 / df1['value1']
df1.head()

In [ ]:
np.mean(df1)

We can also apply functions to the whole dataset or specific columns with the `apply` command. `apply` acts on the whole column at a time (i.e. a Pandas `Series`), so we can compute things that depend on several values of the column, for instance, the mean value. To apply functions in a real element-by-element basis the function `applymap` or `Series.apply` should be used.

In [ ]:
def mean(col):
    return sum(col) / len(col)

df1[['value1', 'value2']].apply(mean)

While most can be directly calculated (including the given example of the mean), `apply` also works on columns with strings or categorical data, where no mathematical operations are defined. The limit is the imagination.

### Combining `DataFrames`

Something we will do quite often as scientists is combining data from different sources into one single source. This can be achieved by different commands in Pandas, depending on the actual goal we want.

To begin with, appending new rows of data is achieved by the command `append`.

In [ ]:
df2 = pd.DataFrame()

df2['sample'] = ['A', 'A', 'A', 'B', 'B', 'B']
df2['replicate'] = ['01', '02', '03', '01', '02', '03']
df2['protein'] = 'P69892'
df2['value1'] = np.random.randn(6)
df2['value2'] = 1 / df2['value1']

df2

In [ ]:
df1.append(df2, ignore_index=True)

The same result can be obtained with `concat`.

In [ ]:
df = pd.concat([df1, df2], ignore_index=True)

df

### Grouping Data

In [ ]:
df.groupby('protein').agg(sum)

In [ ]:
df.groupby(['protein', 'sample']).agg(sum)

In [ ]:
df.groupby(['protein', 'sample', 'replicate']).agg(sum)

In [ ]:
df.groupby('protein').transform(np.mean)

In [ ]:
df.groupby('protein')['value1', 'value2'].transform(np.mean)

In [ ]:
for g, g_df in df.groupby(['protein', 'sample']):
    print(g_df)
    print(f"{g} --> mean value1: {np.mean(g_df['value1'])}")
    print(f"      mean value2: {np.mean(g_df['value2'])}\n")

In [ ]:
df.groupby(['protein', 'sample']).describe()

In [ ]:
df.pivot_table(index='protein',
               columns='sample', 
               aggfunc='mean')

In [ ]:
df.pivot_table(index='protein',
               columns='sample',
               aggfunc={'value1': min,
                        'value2': max})

### Loading and saving dataframes

To load and save Pandas dataframes we will use the `to_csv` and `read_csv` commands. Whenever the dataframe does not contain any kind of column that is of type `object` we can also use feather format with `to_feather`. In case we have objects in the cells, such as functions, for example, we can use pickle format with `to_pickle`. 

In [ ]:
df.to_csv('test.csv')
pd.read_csv('test.csv', index_col=0)

But, as an addition, Pandas has special commands to load and save Excel spreadsheets (yay!). However, to use it you'll need the `openpyxl` and `xlrd` packages.

In [ ]:
df.to_excel('test.xlsx', sheet_name='My sheet')
pd.read_excel('test.xlsx', 'My sheet', index_col=0)

**Exercise 5**: Download [this dataset](https://raw.githubusercontent.com/ChihChengLiang/pokemongor/master/data-raw/pokemons.csv) and load it, using the first column as the index. Take a look at it, and do the following things:
- Choose the columns 'Identifier', 'BaseStamina', 'BaseAttack', 'BaseDefense', 'Type1' and 'Type2' 
- Create a function that lowercases strings and apply it to 'Type1' and 'Type2' (*Extra: just capitalize the strings, i.e., leave the first letter uppercase and lowercase the rest*)
- Create a function that returns a Boolean value (don't be afraif by this, it is a function that returns either True or False) that tells if a Pokémon has high stamina (BaseStamina>170) or not. Store this information in a new column and show the list of Pokémon with high stamina
- Show the instructor the last 15 rows of your dataset

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/ChihChengLiang/pokemongor/master/data-raw/pokemons.csv', 
                 index_col=0)

df = df[['Identifier', 'BaseStamina', 'BaseAttack', 'BaseDefense', 'Type1', 'Type2']]

capitalize = lambda st: st.capitalize()

for col in ['Type1', 'Type2']:
    df[col] = df[col].apply(capitalize)
    
def highstamina(x):
    return True if x > 170 else False

df['HighStamina'] = df.BaseStamina.apply(highstamina)

print(df[df['HighStamina'] == True].Identifier)

df.tail(15)